# Most Important Note : Used for getting data from ony NetApp-DocV3.6

In [ ]:
import chargeback_rpt.db_postgres_command as db_command
import chargeback_rpt.vm_data_utility as vm_util
import chargeback_rpt.vm_data_validator as vx
import chargeback_rpt.email_notifier as x_mail

import chargeback_rpt.file_directory_manager as fd_mn
import pandas as pd
import numpy as np

import os

import openpyxl

import psycopg2
import psycopg2.extras as extras

import datetime
import json

import sys 

# For run script .py

In [ ]:
run_py=False
fix_mmyy=True  #fix_mmyy=True  for test_only_jupyternotebook
is_only_check_data=False

x_datenow=datetime.datetime.now()
month_x=x_datenow.strftime('%m')
year_x=x_datenow.strftime('%Y')

if fix_mmyy:
 month_param='09'
 year_param='2021'
else:
 month_param=month_x
 year_param=year_x
    
    
print(f"Current month-year : {month_param}-{year_param}")

In [ ]:
if  run_py:

    press_y='n'
    ok=False
    
    if len(sys.argv) > 1:
        if sys.argv[1]=='0'  and sys.argv[2]=='0':
         month_param=month_x
         year_param=year_x 
         ok=True   
         is_only_check_data=bool(int(sys.argv[3]))
        else:
         raise Exception("Specify 0 0 as arguments only")   
    else:
        print("If you want current month and year.Press Enter ")
        month_param = int(input("Enter the month (1-12) : ") or month_x)
        year_param = int(input("Enter the year(such as 2021,2022,2023...) : ") or year_x)
        print(f"Do for month={month_param} and year={year_param}")
        press_y=input(f"Press y=True and n=False : ") 
        if press_y.lower()=='y':
         ok=True


    if ok==True:
        try:
         report_dt= datetime.datetime(int(year_param), int(month_param), 1)
         print(report_dt)   
         month_param=report_dt.strftime('%m')
         year_param=report_dt.strftime('%Y')   

    #      n = len(sys.argv) 
    #      print("Total arguments passed:", n) 
    #      for  param in  sys.argv:
    #        print(param)

        except Exception as ex:
         print(ex)   
         raise ex
    else:
         quit()

In [ ]:
errors=[]

size_col='size_used_gb_detail'  # column from sheet  FlexVolSpaceDetails
size_col_2='size_used_gb'        #  column from shee FlexVolConfiguration


VolConfiguration_sheet='netapp_vol'
VolSpaceDetails_sheet='netapp_vol_details'

vol_master_col='volume_name'
vol_detail_col='volume_name_detail'
svm_master_col='svm_name'
svm_detail_col='svm_name_detail'

extend_style='volume_extended_style'

netapp_col_key='netapp'

center_col='cost_center'
system_col='system_name'
created_date_col='created_date'

comment_col='comment'


key_summary='Total' 

total_col_vol='cluster_name'
total_col_detail='cluster_name_detail'

cs_cols_toCheck=[comment_col,center_col,system_col]
#cs_cols_toCheck=[center_col,system_col]

fiter_volDetail =['flexgroup','flexvol']

key_audit_Vol='audit_vol_netapp'


netapp_ok=False
vol_filepath=None

In [ ]:


if is_only_check_data==False:
 t_id=vm_util.creating_transaction(2,month_param,year_param)
 
else:
  t_id=vm_util.creating_transaction(9,month_param,year_param)   
    
print(f"ETL Transaction ID: {t_id}  and OnlyCheckData={is_only_check_data}")    

list_error=[]
print(list_error)

In [ ]:
# Move  to util
def check_error_point_etl(tran_id):
  print(list_error)
  if True in list_error:

        
    etl_files=[]
    if  vol_filepath is not None and os.path.exists(vol_filepath):
     etl_files.append([netapp_ok,vol_filepath])   
    
    if len(etl_files)>0 and (netapp_ok==False) :
     vm_util.finished_etl_folder(t_id,netapp_col_key,False,etl_files)  
        
    vm_util.collect_error_to_sent_mail(tran_id) 
    print("ETL occured error") 
    
    raise Exception("Program is teminated and check error from email and log_error.txt")
    
  list_error.clear()
  print(list_error)   

# Get Any Configvalue

In [ ]:
try:
    audit_vol_netapp=vm_util.get_config_value(key_audit_Vol,t_id)
    print(audit_vol_netapp)
    datetime_format = vm_util.get_config_value("datetime_format", t_id)
    print(datetime_format)
    
except Exception as ex:
    list_error.append(True)
    
check_error_point_etl(t_id)    

# Verify all source files

In [ ]:
print("get root folder")

key_path='import_path'
import_path=vm_util.get_value_by_key(key_path)
print(import_path)

if import_path is not None:
 
 data_import_path=import_path['value']
 existPath,is_error=vx.check_existing_filepath(data_import_path,t_id)
 list_error.append(is_error)   
 if existPath:
  print("Get Path of all source files: ",data_import_path)
else:
 list_error.append(True)   
 error_message= f'no key:{key_path} in key column in config_value table'
 print(error_message)   
 vm_util.add_error_to_database(3,error_message,t_id)  
    
check_error_point_etl(t_id)

In [ ]:
print("get source file name for ready to do ETL")

sr_vol,is_error=vx.check_source_name_file(VolConfiguration_sheet,t_id)
print(sr_vol)
list_error.append(is_error)

sr_detail,is_error=vx.check_source_name_file(VolSpaceDetails_sheet,t_id)
print(sr_detail)
list_error.append(is_error)
check_error_point_etl(t_id)

# Load File

In [ ]:
print("load file and skip some rows")
print(sr_vol)
vol_filepath=f"{data_import_path}\\{sr_vol['source_prefix']}.{sr_vol['source_type']}"
a,is_error=vx.check_existing_filepath(vol_filepath,t_id)
list_error.append(is_error)
print("Get Path of volume file: ",vol_filepath,' is ',a)
print("=======================================================================================================")

check_error_point_etl(t_id)

In [ ]:
skiprows_netapp=vm_util.get_value_by_key('skiprows_netapp')
if skiprows_netapp is not None:
 skiprows=skiprows_netapp['value']
 skiprows_list = list(map(int, skiprows.split(',')))
 print('skip rows netapp: ',skiprows_list)
else:
 list_error.append(True)   
 error_message= f'no key:{key_path} in key column in config_value table'
 print(error_message)   
 vm_util.add_error_to_database(3,error_message,t_id)  


check_error_point_etl(t_id) 

In [ ]:
print("load volume and its detail")

sheet_name_list=[sr_vol['source_keyname'],sr_detail['source_keyname']]
dict_dfs,is_error=vx.load_source_file_as_dict(vol_filepath,sheet_name_list,skiprows_list,t_id)

df_vol=dict_dfs[sr_vol['source_keyname']]
df_detail=dict_dfs[sr_detail['source_keyname']]

check_error_point_etl(t_id) 

print(df_vol.head())
print(df_detail.head())

# Selec required columns from dataframe and Mapping filed from datasouce to column dataframe

In [ ]:
print("Selec required columns from dataframe")
print("Mapping filed from datasouce to column dataframe")


print(f"select columns in {VolConfiguration_sheet} sheet")
df_vol_mappingFields=vm_util.get_active_datafield(sr_vol['id'])

# remove size used gb columne name from main sheet for getting from detail 
print("Remove {size_col_2}")
df_vol_mappingFields=df_vol_mappingFields.query('column_table_name!=@size_col_2')

print(df_vol_mappingFields[['column_table_name','field_source_name','is_additional','datasource_id']])

print("==================FlexVolConfiguration mappingFields==================================")
df_vol,err=vx.select_colunm_df_fieldname_ds(df_vol,df_vol_mappingFields['field_source_name'].tolist(),vol_filepath,t_id,)


print(df_vol.tail())
list_error.append(err)

In [ ]:
print("Mapping FlexVolConfiguration")
df_vol,err=vx.map_fields_ds_to_cols_df(df_vol,df_vol_mappingFields,vol_filepath,t_id)
list_error.append(err)

In [ ]:

print(f"select columns in {VolSpaceDetails_sheet} sheet")
df_detail_mappingFields=vm_util.get_active_datafield(sr_detail['id'])

#replace size column name  in FlexVolSpaceDetails  in order to advoice duplicatetion templorily
df_detail_mappingFields.replace(size_col, size_col_2,  inplace=True)
size_col=size_col_2
print(df_detail_mappingFields[['column_table_name','field_source_name','is_additional','datasource_id']])


print("==================FlexVolSpaceDetails mappingFields==================================")
df_detail,err=vx.select_colunm_df_fieldname_ds(df_detail,df_detail_mappingFields['field_source_name'].tolist(),vol_filepath,t_id,)
print(df_detail.tail())
list_error.append(err)


In [ ]:
print("Mapping FlexVolSpaceDetails")
df_detail,err=vx.map_fields_ds_to_cols_df(df_detail,df_detail_mappingFields,vol_filepath,t_id)
list_error.append(err)


In [ ]:
check_error_point_etl(t_id)


# Remove last row summary  

In [ ]:
print("remove last row summary")  
def remove_summary_row(total_col,dfx):
 idx_last=dfx.tail(-1).query(f'{total_col}==@key_summary').index
 if(len(idx_last)>0):
  dfx.drop(index=idx_last,inplace=True)
 return dfx

In [ ]:
df_vol=remove_summary_row(total_col_vol,df_vol)
df_detail=remove_summary_row(total_col_detail,df_detail)


In [ ]:
print(f"========================={VolConfiguration_sheet}=================================")
print(df_vol.info())
print(df_vol.head())
print(df_vol.tail())
print(f"========================={VolSpaceDetails_sheet}=================================")
print(df_detail.info())
print(df_detail.head())
print(df_detail.tail())

In [ ]:
check_error_point_etl(t_id) 


# Filter Volumn Main

In [ ]:
print(f"Filter Volume Name By Excluding AuditVolume started with  {audit_vol_netapp}")

df_vol = df_vol[df_vol[vol_master_col].apply(lambda volName: not volName.lower().startswith(audit_vol_netapp.lower()))] 
print(df_vol.head())
print(df_vol.tail())

# Filter Volumn Detail 

In [ ]:
print("Filter Volume Extended Style only ",fiter_volDetail)
df_detail=df_detail.query('volume_extended_style in @fiter_volDetail').sort_values(by=vol_detail_col, ascending=False)

#print(df_detail[vol_detail_col].unique())

print(df_detail.head())
print(df_detail.tail())

# Merge Volumn and its Detail

In [ ]:
print("Merge Volumn and its Detail")
try:
     
#  df_vol=df_vol.merge(df_detail[[vol_detail_col,size_col]],how='left',left_on=vol_master_col,right_on=vol_detail_col)
#  df_vol.drop(columns=[vol_detail_col],inplace=True)
   

 df_vol=df_vol.merge(df_detail[[svm_detail_col,vol_detail_col,size_col]], 
            how='left',left_on=[vol_master_col,svm_master_col],right_on=[vol_detail_col,svm_detail_col])
 df_vol.drop(columns=[vol_detail_col,svm_detail_col],inplace=True)
  
        
except Exception as err:
 list_error.append(True)
 error_message= str(err)
 print(error_message)   
 vm_util.add_error_to_database(14,error_message,t_id) 

In [ ]:

print(df_vol)

In [ ]:
check_error_point_etl(t_id) 

# Transform &  Extract Volumn Info

# Extract Cost Center and System Name and CreatedDate


In [ ]:
print("Extract Cost Center and System Name and Created Date")
df_vol[center_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(center_col,)) 
df_vol[system_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(system_col,))
df_vol[created_date_col]=df_vol[comment_col].apply(vm_util.split_comment_to_each_value,args=(created_date_col,))
print(df_vol.head(10))

In [ ]:
print("Check invalid  data after extracting comment")
cols_na_comment,df_na_comment=vx.find_NaN(df_vol,cs_cols_toCheck)

print(cols_na_comment)
print(df_na_comment)

if df_na_comment.shape[0]>0:
    
 str_error = '</br>Summarize rows that found any null or incorrect format string in <b><u>comment column</u></b></br>'
 df_na_comment= df_na_comment.fillna("")
 str_error += df_na_comment.to_html(index=False)
 vm_util.add_error_to_database(16, str_error,t_id)
 list_error.append(True)   


check_error_point_etl(t_id) 
print(df_vol.info())
print(df_vol)

# Convert string date to datetime

In [ ]:
import calendar
lastDay_val=datetime.date(int(year_param),int(month_param),calendar.monthrange(int(year_param), int(month_param))[1])
print(lastDay_val)
print("Validate CreatedDate")

df_vol=vx.find_incorrrect_format_for_input_datetime(df_vol,created_date_col,vol_master_col,datetime_format,t_id)
if df_vol is None:
 list_error.append(True)
 print(list_error)
else:    
 df_vol=vx.find_invalid_x_date_greater_last_date_for_input_datetime(df_vol,created_date_col,lastDay_val,vol_master_col,t_id)
 if df_vol is None:
  list_error.append(True) 
  print(list_error)
 else:
   print(f"all created_date > {lastDay_val}")   
   print( df_vol[[vol_master_col,created_date_col]] ) 
    
check_error_point_etl(t_id)
    

# Transform  additional data

In [ ]:
print("# Transform addtoinal data")
try:
     

 df_vol['month']=month_param
 df_vol['year']=year_param
 df_vol['import_date']=x_datenow


 df_vol['transaction_id']= t_id

 #df_vol=df_vol.where(pd.notnull(df_vol), None) # replace nan with None then call replace_x_character
 df_vol['system_name']=df_vol['system_name'].apply(vx.replace_x_character)
 df_vol['cost_center']=df_vol['cost_center'].apply(vx.replace_x_character)   


except Exception as err:
  list_error.append(True)
  error_message= str(err)
  print(error_message)   
  vm_util.add_error_to_database(14,error_message,t_id)  
  


In [ ]:
check_error_point_etl(t_id) 
netapp_ok=True

print("Last Check  prior to saving to database")
print(df_vol.info())
print(df_vol[[vol_master_col,size_col,center_col,system_col,'created_date','month','year','import_date']])

# End up with  Completed ETL in only Checking Data mode

In [ ]:
if is_only_check_data==True:
    try: 

        delete_resultList= fd_mn.delele_file(vol_filepath)   
        updated_rows=vm_util.created_transaction(t_id)
        print("completed ETL in only Checking Data mode")
        
        exit()
        
        
    except Exception as ex:
        list_error.append(True)
        print(ex)
        raise Exception("Program is teminated and check error from email and log_error.txt")
    

# Add data to table

In [ ]:
print("Check columns matching")

In [ ]:
vm_tb_key="storage_table"
table_name,listCols_report,is_error=vx.listCols_report_table(vm_tb_key,t_id)
list_error.append(is_error)

check_error_point_etl(t_id)
print(f'list all columns in {table_name} table: ',len(listCols_report))
print(listCols_report)


print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
listCols_dfx=df_vol.columns.tolist()
print(f'list all columns in {table_name} dataframe: ',len(listCols_dfx))
print(listCols_dfx)

In [ ]:
check_columns_2=vm_util.verify_existing_columns(listCols_dfx,listCols_report)

print(check_columns_2)
listCols_report

In [ ]:
print("Save data to database")
if check_columns_2  is None:
   df_vol=df_vol[listCols_report]
   print(df_vol.info())
   #df_vol.to_excel('report_vol_before_DB.xlsx')
   print(df_vol.head())
   rslt=db_command.add_data_values(db_command.get_postgres_conn(),df_vol,table_name)
   #print(rslt)
    
else:
   list_error.append(True)
   error_message=check_columns_2
   vm_util.add_error_to_database(10,error_message,t_id)
   print(error_message)


In [ ]:
check_error_point_etl(t_id)

# Update new cost center

In [ ]:
print("Update new cost center")

try :
 cost_centerList = df_vol['cost_center'].unique()
 df_new_cc=vm_util.add_new_cost_center(cost_centerList,'NetApp-ETL',t_id)
 print(df_new_cc)
except Exception as ex:
 list_error.append(True)   
 print(ex)   

check_error_point_etl(t_id)


# Completed ETL

In [ ]:
try: 
    etl_files=[[netapp_ok,vol_filepath]]
    if netapp_ok==True :
     vm_util.finished_etl_folder(t_id,netapp_col_key,True,etl_files)    
     updated_rows=vm_util.created_transaction(t_id)
     print("completed ETL")
    
except Exception as ex:
    list_error.append(True)
    print(ex)
    
check_error_point_etl(t_id)